In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mahdifaour/heart-disease-dataset")

print("Path to dataset files:", path)

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/morning/.cache/kagglehub/datasets/mahdifaour/heart-disease-dataset/versions/1


In [2]:
import os
os.listdir(path)

['Heart_Disease (1).csv']

In [3]:
import pandas as pd
import fairtl as fl
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [4]:
df = pd.read_csv(f'{path}/Heart_Disease (1).csv')
df.head()

,sex,age,education,smokingStatus,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,CHDRisk
0,male,39,4,no,0,0,0,0,no,195,106.0,70.0,26.97,80,77,no
1,female,46,2,no,0,0,0,0,no,250,121.0,81.0,28.73,95,76,no
2,male,48,1,yes,20,0,0,0,no,245,127.5,80.0,25.34,75,70,no
3,female,61,3,yes,30,0,0,1,no,225,150.0,95.0,28.58,65,103,yes
4,female,46,3,yes,23,0,0,0,no,285,130.0,84.0,23.10,85,85,no


In [5]:
df.shape

(3674, 16)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3674 entries, 0 to 3673
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sex              3663 non-null   object 
 1   age              3674 non-null   int64  
 2   education        3674 non-null   int64  
 3   smokingStatus    3661 non-null   object 
 4   cigsPerDay       3674 non-null   int64  
 5   BPMeds           3674 non-null   int64  
 6   prevalentStroke  3674 non-null   int64  
 7   prevalentHyp     3674 non-null   int64  
 8   diabetes         3674 non-null   object 
 9   totChol          3674 non-null   int64  
 10  sysBP            3674 non-null   float64
 11  diaBP            3674 non-null   float64
 12  BMI              3674 non-null   float64
 13  heartRate        3674 non-null   int64  
 14  glucose          3674 non-null   int64  
 15  CHDRisk          3674 non-null   object 
dtypes: float64(3), int64(9), object(4)
memory usage: 459.4+ KB


In [7]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3652 entries, 0 to 3651
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sex              3652 non-null   object 
 1   age              3652 non-null   int64  
 2   education        3652 non-null   int64  
 3   smokingStatus    3652 non-null   object 
 4   cigsPerDay       3652 non-null   int64  
 5   BPMeds           3652 non-null   int64  
 6   prevalentStroke  3652 non-null   int64  
 7   prevalentHyp     3652 non-null   int64  
 8   diabetes         3652 non-null   object 
 9   totChol          3652 non-null   int64  
 10  sysBP            3652 non-null   float64
 11  diaBP            3652 non-null   float64
 12  BMI              3652 non-null   float64
 13  heartRate        3652 non-null   int64  
 14  glucose          3652 non-null   int64  
 15  CHDRisk          3652 non-null   object 
dtypes: float64(3), int64(9), object(4)
memory usage: 456.6+ KB


In [8]:
df.head()

,sex,age,education,smokingStatus,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,CHDRisk
0,male,39,4,no,0,0,0,0,no,195,106.0,70.0,26.97,80,77,no
1,female,46,2,no,0,0,0,0,no,250,121.0,81.0,28.73,95,76,no
2,male,48,1,yes,20,0,0,0,no,245,127.5,80.0,25.34,75,70,no
3,female,61,3,yes,30,0,0,1,no,225,150.0,95.0,28.58,65,103,yes
4,female,46,3,yes,23,0,0,0,no,285,130.0,84.0,23.10,85,85,no


In [9]:
df['sex'] = df['sex'].map({'female': 0, 'male': 1})
df['smokingStatus'] = df['smokingStatus'].map({'no': 0, 'yes': 1})
df['diabetes'] = df['diabetes'].map({'no': 0, 'yes': 1})
df['CHDRisk'] = df['CHDRisk'].map({'no': 0, 'yes': 1})


In [10]:
df.head()

,sex,age,education,smokingStatus,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,CHDRisk
0,1,39,4,0,0,0,0,0,0,195,106.0,70.0,26.97,80,77,0
1,0,46,2,0,0,0,0,0,0,250,121.0,81.0,28.73,95,76,0
2,1,48,1,1,20,0,0,0,0,245,127.5,80.0,25.34,75,70,0
3,0,61,3,1,30,0,0,1,0,225,150.0,95.0,28.58,65,103,1
4,0,46,3,1,23,0,0,0,0,285,130.0,84.0,23.10,85,85,0


In [11]:
y = df['CHDRisk'].values
X = df.drop('CHDRisk', axis=1)

In [12]:
print(X.shape)
selector = VarianceThreshold(threshold=0.1)
X = selector.fit_transform(X)
print(X.shape)

(3652, 15)
(3652, 12)


In [13]:
# build mask
gender_0_mask = df['sex'] == 0
gender_1_mask = df['sex'] == 1

count_gender_0 = gender_0_mask.sum()
count_gender_1 = gender_1_mask.sum()

print("Female: ", count_gender_0)
print("Male: ", count_gender_1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_Gender_0 = X_scaled[gender_0_mask]
X_scaled_Gender_1 = X_scaled[gender_1_mask]
y_Gender_0 = y[gender_0_mask]
y_Gender_1 = y[gender_1_mask]

Female:  2026
Male:  1626


In [14]:
seed = 42
models = fl.build_models(seed)

results_list = []

kf = KFold(n_splits=10, shuffle=True, random_state=seed)

In [15]:
print("Starting experiments for Female(0)")
fl.run_experiment(kf, models, X_scaled_Gender_0, y_Gender_0, 'Female', results_list)

print("Starting experiments for Male(1)")
fl.run_experiment(kf, models, X_scaled_Gender_1, y_Gender_1, 'Male', results_list)

final_results_df = pd.concat(results_list, ignore_index=True)
print(final_results_df)

Starting experiments for Female(0)
Processing fold 1 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 2 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 3 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


...

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 9 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 10 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
    Fold   Group   SVM_TPR   SVM_TNR   SVM_FPR   SVM_FNR  SVM_TP  SVM_TN  \
0      1  Female  0.000000  1.000000  0.000000  1.000000       0     178   
1      2  Female  0.000000  0.994565  0.005435  1.000000       0     183   
2      3  Female  0.068966  1.000000  0.000000  0.931034       2     174   
3      4  Female  0.000000  1.000000  0.000000  1.000000       0     181   
4      5  Female  0.000000  1.000000  0.000000  1.000000       0     181   
5      6  Female  0.000000  1.000000  0.000000  1.000000       0     179   
6      7  Female  0.000000  1.000000  0.000000  1.000000       0     169   
7      8  Female  0.000000  1.000000  0.000000  1.000000       0     175   
8      

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [16]:
results_df = pd.concat(results_list, ignore_index=True)

In [17]:
result_path = './results/k51_result.xlsx'
results_df.to_excel(result_path, index=False)

In [18]:
df = pd.read_excel(result_path)
df.head()

,Fold,Group,SVM_TPR,SVM_TNR,SVM_FPR,SVM_FNR,SVM_TP,SVM_TN,SVM_FP,SVM_FN,...,ANN_FP,ANN_FN,NB_TPR,NB_TNR,NB_FPR,NB_FNR,NB_TP,NB_TN,NB_FP,NB_FN
0,1,Female,0.000000,1.000000,0.000000,1.000000,0,178,0,25,...,5,24,0.400000,0.915730,0.084270,0.600000,10,163,15,15
1,2,Female,0.000000,0.994565,0.005435,1.000000,0,183,1,19,...,6,17,0.263158,0.847826,0.152174,0.736842,5,156,28,14
2,3,Female,0.068966,1.000000,0.000000,0.931034,2,174,0,27,...,5,26,0.310345,0.862069,0.137931,0.689655,9,150,24,20
3,4,Female,0.000000,1.000000,0.000000,1.000000,0,181,0,22,...,3,19,0.363636,0.922652,0.077348,0.636364,8,167,14,14
4,5,Female,0.000000,1.000000,0.000000,1.000000,0,181,0,22,...,2,20,0.318182,0.922652,0.077348,0.681818,7,167,14,15


In [19]:
label = 'Female'

fl.perform_t_tests(df, 'SVM', label)
fl.perform_t_tests(df, 'DT', label)
fl.perform_t_tests(df, 'RF', label)
fl.perform_t_tests(df, 'LR', label)
fl.perform_t_tests(df, 'KNN', label)
fl.perform_t_tests(df, 'ANN', label)
fl.perform_t_tests(df, 'NB', label)


SVM -TPR: MannwhitneyuResult(statistic=np.float64(24.0), pvalue=np.float64(0.03943628317378319))
SVM - FPR: MannwhitneyuResult(statistic=np.float64(30.0), pvalue=np.float64(0.08369666249056641))
SVM - FN/FP: MannwhitneyuResult(statistic=np.float64(49.5), pvalue=np.float64(1.0))
DT -TPR: TtestResult(statistic=np.float64(-2.848987974676719), pvalue=np.float64(0.010654919760909006), df=np.float64(18.0))
DT - FPR: TtestResult(statistic=np.float64(-4.236206866566623), pvalue=np.float64(0.0004966374144444196), df=np.float64(18.0))
DT - FN/FP: TtestResult(statistic=np.float64(-0.6229169552986179), pvalue=np.float64(0.5411516982798489), df=np.float64(18.0))
RF -TPR: MannwhitneyuResult(statistic=np.float64(15.5), pvalue=np.float64(0.010052095939085178))
RF - FPR: TtestResult(statistic=np.float64(-1.9275523649011344), pvalue=np.float64(0.0698423145856973), df=np.float64(18.0))
RF - FN/FP: MannwhitneyuResult(statistic=np.float64(49.0), pvalue=np.float64(0.9698499769931556))
LR -TPR: MannwhitneyuR